explore data

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
df_train = pd.read_csv('../input/train.csv.zip',compression='zip')
df_per_train = pd.read_csv('../input/periods_train.csv.zip',compression='zip')


In [13]:
df_test = pd.read_csv('../input/test.csv.zip',compression='zip')

size 

In [8]:
df_train.shape

(1503424, 18)

In [10]:
df_per_train.shape

(16687412, 4)

In [15]:
df_test.shape

(508438, 17)

In [12]:
df_train.head().T

,0,1,2,3,4
item_id,b912c3c6a6ad,2dac0150717d,ba83aefab5dc,02996f1dd2ea,7c90be56d2ab
user_id,e00f8ff2eaf9,39aeb48f0017,91e2f88dd6e3,bf5cccea572d,ef50846afc0b
region,Свердловская область,Самарская область,Ростовская область,Татарстан,Волгоградская область
city,Екатеринбург,Самара,Ростов-на-Дону,Набережные Челны,Волгоград
parent_category_name,Личные вещи,Для дома и дачи,Бытовая электроника,Личные вещи,Транспорт
category_name,Товары для детей и игрушки,Мебель и интерьер,Аудио и видео,Товары для детей и игрушки,Автомобили
param_1,Постельные принадлежности,Другое,"Видео, DVD и Blu-ray плееры",Автомобильные кресла,С пробегом
param_2,NaN,NaN,NaN,NaN,ВАЗ (LADA)
param_3,NaN,NaN,NaN,NaN,2110
title,Кокоби(кокон для сна),Стойка для Одежды,Philips bluray,Автокресло,"ВАЗ 2110, 2003"


- number of unique uid

In [16]:
df_train.user_id.nunique() # 771,769 out of 1,503,424 

771769

In [18]:
df_test.user_id.nunique() # 306,069 

306069

In [28]:
uid_in_trn_test = np.intersect1d(df_test.user_id,df_train.user_id) # 67,929

In [31]:
pd.Series(uid_in_trn_test).nunique()

67929

In [32]:
test_uid = df_test.user_id.unique()
train_uid = df_train.user_id.unique()

In [36]:
771769- 67929

703840

In [37]:
306069 - 67929

238140

- number of unique itemid

In [40]:
train_itemid = df_train.item_id.unique()
test_itemid = df_test.item_id.unique()

In [41]:
train_itemid.size

1503424

In [42]:
test_itemid.size

508438

In [43]:
itmid_in_trn_test = np.intersect1d(train_itemid,test_itemid)

In [44]:
itmid_in_trn_test.size

0

**itemid** is pkey in table 